In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
from PIL import Image # Import PIL library

# Load Caltech101 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda image: image.convert("RGB")),  # Convert to RGB
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.Caltech101(root="./data", download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define Variational Pruning Layer
class VariationalPruneLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features))
        self.weight_rho = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias = nn.Parameter(torch.Tensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_normal_(self.weight_mu)
        nn.init.constant_(self.weight_rho, -5)
        nn.init.constant_(self.bias, 0)

    def forward(self, x):
        weight_sigma = torch.log1p(torch.exp(self.weight_rho))  # Ensure positivity
        weight_sample = self.weight_mu + weight_sigma * torch.randn_like(self.weight_mu)
        return F.linear(x, weight_sample, self.bias)

# Load Pretrained ResNet-50 and Modify Last Layer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)

# Replace last fully connected layer with Variational Pruning Layer
in_features = model.fc.in_features
model.fc = VariationalPruneLayer(in_features, 101)  # 101 classes in Caltech101
model = model.to(device)

# Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Train Model
for epoch in range(15):  # Train for 15 epochs
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/15], Loss: {loss.item():.4f}")

# Save Model
torch.save(model.state_dict(), "resnet50_variational_pruning.pth")

Files already downloaded and verified


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/15], Loss: 0.9923
Epoch [2/15], Loss: 1.2376
Epoch [3/15], Loss: 0.6969
Epoch [4/15], Loss: 0.1419
Epoch [5/15], Loss: 0.3612
Epoch [6/15], Loss: 0.2825
Epoch [7/15], Loss: 0.0357
Epoch [8/15], Loss: 0.0460
Epoch [9/15], Loss: 0.0769
Epoch [10/15], Loss: 0.3764
Epoch [11/15], Loss: 0.3040
Epoch [12/15], Loss: 0.1145
Epoch [13/15], Loss: 0.1168
Epoch [14/15], Loss: 0.1113
Epoch [15/15], Loss: 0.4297


In [3]:
# Load Test Dataset
test_dataset = datasets.Caltech101(root="./data", download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluate Model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Final Accuracy: {accuracy:.2f}%")

Files already downloaded and verified
Final Accuracy: 99.31%
